In [3]:
from sagemaker.workflow.function_step import step
from sagemaker.workflow.pipeline import Pipeline
import sagemaker
from sagemaker.workflow.parameters import ParameterInteger
import utils

# FUNCIONES

In [4]:
user = utils.get_username()
pipeline_name = f"pipeline-basico-{user}"
role = sagemaker.get_execution_role()
#instance_type = "ml.m5.xlarge"
#instance_type = "ml.t3.medium"
instance_type = "ml.m5.large"
periodo = ParameterInteger(name="PeriodoCarga", default_value=202503)

In [5]:
@step(
    name="DataPull",
    instance_type=instance_type,
)
def data_pull(data_source: str, periodo: int) -> tuple[str, str]:
    print(f"Data pull from {data_source}")
    print(f"Periodo:{periodo}")
    return "s3://train_path", "s3://test_path"

In [6]:
@step(
    name="ModelTraining",
    instance_type=instance_type,
)
def model_training(train_path: str, test_path: str, periodo: int) -> str:
    print(f"Train set: {train_path} and test set:{test_path}")
    print(f"Periodo:{periodo}")
    return "model_path"

In [7]:
data_pull_step = data_pull("s3:mybucket/data", periodo)
model_training_step = model_training(data_pull_step[0],
                                     data_pull_step[1], periodo)

In [8]:
pipeline = Pipeline(name=pipeline_name, steps=[data_pull_step, model_training_step],parameters=[periodo])
pipeline.upsert(role_arn=role)

2025-06-05 03:49:44,145 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-654654589924/pipeline-basico-ricardo-puma/DataPull/2025-06-05-03-49-43-898/function
2025-06-05 03:49:44,225 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-654654589924/pipeline-basico-ricardo-puma/DataPull/2025-06-05-03-49-43-898/arguments
2025-06-05 03:49:44,723 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-654654589924/pipeline-basico-ricardo-puma/ModelTraining/2025-06-05-03-49-43-898/function
2025-06-05 03:49:44,820 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-654654589924/pipeline-basico-ricardo-puma/ModelTraining/2025-06-05-03-49-43-898/arguments


{'PipelineArn': 'arn:aws:sagemaker:us-east-1:654654589924:pipeline/pipeline-basico-ricardo-puma',
 'ResponseMetadata': {'RequestId': '0a953af1-e95c-470f-97b6-de381781d28c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0a953af1-e95c-470f-97b6-de381781d28c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '96',
   'date': 'Thu, 05 Jun 2025 03:49:45 GMT'},
  'RetryAttempts': 0}}

In [9]:
pipeline.start(parameters={"PeriodoCarga":202505})

_PipelineExecution(arn='arn:aws:sagemaker:us-east-1:654654589924:pipeline/pipeline-basico-ricardo-puma/execution/ml18fm4wl7ca', sagemaker_session=<sagemaker.session.Session object at 0x7f364755c0e0>)

In [ ]:
#execution = pipeline.start()
#execution.describe()
#execution.wait()
#execution.list_steps()